## Capture images

In [ ]:
import time
import cv2
import datetime as dt
from notify_run import Notify
notify = Notify()

In [ ]:
starttime = time.time()
notification_sent = False
ramp_frames = 10
while True:
    try:
        cap = cv2.VideoCapture(0) # video capture source camera (Here webcam of laptop) 
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        for i in range(ramp_frames):
            ret,frame = cap.read()
            time.sleep(1/ramp_frames)
        cv2.imwrite(f'images/capture_{dt.datetime.now().strftime("%Y%m%d_%H%M%S")}.png',frame)
        cap.release()
        cv2.destroyAllWindows()
    except:
        if not notification_sent:
            notify.send('Timelapse has run into an error')
            notification_sent=True
        
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))